# Python

In [1]:
import glob
import os

In [2]:
!pwd

/lustre1/project/stg_00090/scatac_benchmark/full_4_merged


In [3]:
scrub_name_suffix = "0-4"
loom_path_dict  = {x.split('/')[-1].split(f'.scr')[0]:x for x in sorted(glob.glob(f'cell_region_looms/*singlets.ID.loom'))}
loom_path_dict

{'master_all_1.FULL': 'cell_region_looms/master_all_1.FULL.scrublet0-4.singlets.ID.loom'}

In [4]:
frags_path_dict = {'master_all_1.FULL': '/lustre1/project/stg_00090/scatac_benchmark/full_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz'}

In [5]:
!cat ../0_resources/scripts/seurat_label_transfer_consensus.R

#!/usr/bin/env Rscript
library(Seurat)
library(SeuratDisk)
library(Signac)
# library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(stringr)

args = commandArgs(trailingOnly=TRUE)
sample_id = args[1]
f_loom = args[2]
f_frag = args[3]
f_reference = args[4]
f_annotation = args[5]
f_out = args[6]

print(paste0("Processing sample ", args[1]))

# load pbmc object
# pbmc.rna <- readRDS("../0_resources/seurat_references/pbmc_integrated.RDS")
#pbmc.rna <- readRDS('/lustre1/project/stg_00090/scatac_benchmark/0_resources/seurat_references/pbmc_ssc_mat__integrated.rds')

################################################################################
# ATAC
################################################################################

### get data from loom:
atacloomcon <- Connect(filename = f_loom, mode = "r")
atacloomcon
atac_tmp <- as.Seurat(atacloomcon, assay='ATAC')
atacloomcon$close_all()

# subset by removing contig chromosomes
rawregions = rownames(GetAssayData(atac_tmp, slot = "counts",

```
conda activate r4_2
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/lustre1/project/stg_00002/lcb/fderop/software_2/anaconda3/lib"
```

In [6]:
parallel_filename = "seurat_label_transfer.parallel"
script_path = "../0_resources/scripts/seurat_label_transfer_consensus.R"
img_path = "../0_resources/vsn_cache/cflerin-seurat-4.0.3-plus.sif"
reference_path = "../0_resources/seurat_references/pbmc_ref.rds"
annotation_path = "../0_resources/seurat_references/granges_annotation.rds"
bind_mounts = "/dodrio,/readonly/dodrio,/tmp"
#bind_mounts = "/lustre1,/staging,${VSC_SCRATCH}/tmp:/tmp"

with open(parallel_filename, 'w') as f:
    for sample in loom_path_dict.keys():
        outfile = f"cell_type_classification/{sample}__cell_type_seurat.txt"
        if not os.path.exists(outfile):
            loomfile = loom_path_dict[sample]
            fragfile = frags_path_dict[sample]
            command=f"Rscript {script_path} {sample} {loomfile} {fragfile} {reference_path} {annotation_path} {outfile}"
            f.write(f"{command}\n")
            print(command)

        else:
            f.write(f"#{outfile} already exists!\n")
            print(f"#{outfile} already exists!")

Rscript ../0_resources/scripts/seurat_label_transfer_consensus.R master_all_1.FULL cell_region_looms/master_all_1.FULL.scrublet0-4.singlets.ID.loom /lustre1/project/stg_00090/scatac_benchmark/full_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz ../0_resources/seurat_references/pbmc_ref.rds ../0_resources/seurat_references/granges_annotation.rds cell_type_classification/master_all_1.FULL__cell_type_seurat.txt


In [7]:
!pwd

/lustre1/project/stg_00090/scatac_benchmark/full_4_merged


# Run the Rscript

```
my_conda_activate
conda activate r4_2
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/lustre1/project/stg_00002/lcb/fderop/software_2/anaconda3/lib"
Rscript ../0_resources/scripts/seurat_label_transfer_consensus.R master_all_1.FULL cell_region_looms/master_all_1.FULL.scrublet0-4.singlets.ID.loom /lustre1/project/stg_00090/scatac_benchmark/full_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz ../0_resources/seurat_references/pbmc_ref.rds ../0_resources/seurat_references/granges_annotation.rds cell_type_classification/master_all_1.FULL__cell_type_seurat.txt

```